In [1]:
import torch
import numpy as np
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import random_split, DataLoader
import torchvision.datasets as datasets
import art.attacks.evasion as toolbox
from art.estimators.classification import PyTorchClassifier
from tqdm import tqdm
from art.utils import load_cifar10

d:\Arcade Projects\Knowledge_Distillation_Testing\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("Check current device: ")
# Check if GPU is available, and if not, use the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available(): # Should return True
    print(f"Using GPU: {torch.cuda.get_device_name(0)}") # Should show your GPU name
else:
    print("Using CPU")

### set RNG seed
RNG_SEED = 0

np.random.seed(RNG_SEED)
torch.manual_seed(RNG_SEED)
if torch.cuda.is_available():
	torch.cuda.manual_seed(RNG_SEED)
	torch.cuda.manual_seed_all(RNG_SEED)

Check current device: 
Using GPU: NVIDIA GeForce RTX 4060


In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

test_images, test_labels = next(iter(test_loader))
test_images = test_images.numpy()  # Convert images to NumPy
test_labels = test_labels.numpy()  # Convert labels to NumPy


Files already downloaded and verified


In [4]:
eps_res = 18
eps_min = 0
eps_max = 0.005

# Generating the list of values
eps_list = [eps_min + i * (eps_max - eps_min) / (eps_res - 1) for i in range(eps_res)]
noise_percents = np.arange(0.0, 0.55, 0.05)  # Noise levels from 0.0 to 0.5

files = ['MobileNetV3_large_CE.pth', 'MobileNetV3_small_10_CE.pth', 
         'KD_model_2_0.pth', 'KD_model_2_1.pth', 'KD_model_2_2.pth', 'KD_model_2_3.pth', 'KD_model_2_4.pth', 
         'KD_model_3_5.pth', 'KD_model_3_6.pth', 'KD_model_3_7.pth', 'KD_model_3_8.pth', 'KD_model_3_9.pth',
         'KD_model_4_10.pth', 'KD_model_4_11.pth', 'KD_model_4_12.pth', 'KD_model_4_13.pth', 'KD_model_4_14.pth',
         'KD_model_5_15.pth', 'KD_model_5_16.pth', 'KD_model_5_17.pth', 'KD_model_5_18.pth', 'KD_model_5_19.pth']

pgd_all_accuracies = []
snp_all_accuracies = []

In [5]:
def toolbox_generate(eps, classifier, test_image=test_images, test_label=test_labels):
    attack = toolbox.ProjectedGradientDescent(estimator=classifier, eps=eps)
    x_test_adv = attack.generate(x=test_image)
    # Step 7: Evaluate the ART classifier on adversarial test examples
    predictions = classifier.predict(x_test_adv)
    accuracy = np.sum(np.argmax(predictions, axis=1) == test_label) / len(test_label)
    return accuracy

In [6]:
# Salt-and-Pepper Noise Class (same as corrected version)
class SaltAndPepperNoise:
    def __init__(self, noise_percent=0.1):
        self.noise_percent = noise_percent

    def generate(self, xs):
        noisy_xs = np.copy(xs)
        num_pixels = int(self.noise_percent * xs.size)

        # Salt noise
        salt_coords = [np.random.randint(0, i, num_pixels // 2) for i in xs.shape]
        noisy_xs[tuple(salt_coords)] = 1.0

        # Pepper noise
        pepper_coords = [np.random.randint(0, i, num_pixels // 2) for i in xs.shape]
        noisy_xs[tuple(pepper_coords)] = 0.0

        return noisy_xs

In [7]:
# Define loss function and optimizer
for f in files:  
    if f == 'MobileNetV3_large_CE.pth':
      torch.manual_seed(42)
      model = models.mobilenet_v3_large(weights=None)
    else:
      torch.manual_seed(42)
      model = models.mobilenet_v3_small(weights=None)

    model.classifier[3] = nn.Linear(model.classifier[3].in_features, 10)
    model.load_state_dict(torch.load(f, map_location=device, weights_only=True))
    model.eval()
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    classifier_pgd = PyTorchClassifier(
        model=model,
        loss=loss_fn,
        optimizer=optimizer,
        input_shape=(3, 224, 224), 
        nb_classes=10,
        device_type="gpu"
    )

    classifier_snp = PyTorchClassifier(
        model=model,
        loss=loss_fn,
        optimizer=optimizer,
        input_shape=(3, 224, 224), 
        nb_classes=10,
        device_type="gpu"
    )

    pgd_accuracy = []
    snp_accuracy = []

    print("--------------------------")
    print(f"Running PGD......... {f}")
    for eps in eps_list:
        accuracy = toolbox_generate(eps, classifier_pgd)
        pgd_accuracy.append(accuracy * 100)
        print(f"Accuracy with epsilon {eps}: {accuracy * 100:.2f}%")
    pgd_all_accuracies.append(pgd_accuracy)

    print(f"Running SNP......... {f}")

    # Apply Salt-and-Pepper Noise for each noise level
    for noise_percent in noise_percents:
        print(f"\nEvaluating Salt-and-Pepper Noise with noise_percent = {noise_percent}")
        spn_attack = SaltAndPepperNoise(noise_percent=noise_percent)

        # Initialize lists to collect noisy images and labels
        noisy_test_images = []
        test_labels = []

        # Generate noisy images for the entire test set
        for images, labels in tqdm(test_loader, desc=f"Adding Noise (percent={noise_percent})"):
            noisy_images = spn_attack.generate(images.numpy())
            noisy_test_images.append(noisy_images)
            test_labels.append(labels.numpy())

        # Concatenate all noisy examples and labels
        noisy_test_images = np.vstack(noisy_test_images)
        test_labels = np.hstack(test_labels)

        # Predict on noisy examples
        preds = np.argmax(classifier_snp.predict(noisy_test_images), axis=1)

        # Calculate accuracy
        accuracy = np.mean(preds == test_labels) * 100
        print(f"Accuracy with noise_percent={noise_percent}: {accuracy:.2f}%")

        # Store the results
        snp_accuracy.append(accuracy)
    snp_all_accuracies.append(snp_accuracy)
    

--------------------------
Running PGD......... MobileNetV3_large_CE.pth


Accuracy with epsilon 0.0: 84.38%


Accuracy with epsilon 0.00029411764705882356: 84.38%


Accuracy with epsilon 0.0005882352941176471: 78.12%


Accuracy with epsilon 0.0008823529411764705: 81.25%


...

Accuracy with noise_percent=0.4: 11.39%

Evaluating Salt-and-Pepper Noise with noise_percent = 0.45


Adding Noise (percent=0.45): 100%|██████████| 313/313 [00:31<00:00,  9.95it/s]


Accuracy with noise_percent=0.45: 11.08%

Evaluating Salt-and-Pepper Noise with noise_percent = 0.5


Adding Noise (percent=0.5): 100%|██████████| 313/313 [00:34<00:00,  9.16it/s]


Accuracy with noise_percent=0.5: 10.81%


In [8]:
print(pgd_all_accuracies)

[[np.float64(84.375), np.float64(84.375), np.float64(78.125), np.float64(81.25), np.float64(78.125), np.float64(71.875), np.float64(68.75), np.float64(62.5), np.float64(62.5), np.float64(62.5), np.float64(56.25), np.float64(50.0), np.float64(50.0), np.float64(43.75), np.float64(43.75), np.float64(40.625), np.float64(40.625), np.float64(37.5)], [np.float64(90.625), np.float64(87.5), np.float64(75.0), np.float64(71.875), np.float64(71.875), np.float64(71.875), np.float64(68.75), np.float64(68.75), np.float64(62.5), np.float64(62.5), np.float64(56.25), np.float64(56.25), np.float64(53.125), np.float64(46.875), np.float64(43.75), np.float64(43.75), np.float64(43.75), np.float64(43.75)], [np.float64(93.75), np.float64(93.75), np.float64(90.625), np.float64(84.375), np.float64(81.25), np.float64(78.125), np.float64(68.75), np.float64(65.625), np.float64(62.5), np.float64(59.375), np.float64(56.25), np.float64(53.125), np.float64(46.875), np.float64(46.875), np.float64(40.625), np.float64(37.

In [9]:
print(snp_all_accuracies)

[[np.float64(84.95), np.float64(72.45), np.float64(61.129999999999995), np.float64(50.67), np.float64(42.18), np.float64(36.29), np.float64(30.9), np.float64(26.87), np.float64(23.49), np.float64(21.45), np.float64(19.919999999999998)], [np.float64(83.81), np.float64(65.95), np.float64(50.629999999999995), np.float64(39.14), np.float64(32.34), np.float64(27.21), np.float64(24.0), np.float64(21.58), np.float64(19.650000000000002), np.float64(18.32), np.float64(16.93)], [np.float64(84.48), np.float64(63.36000000000001), np.float64(46.45), np.float64(35.24), np.float64(27.92), np.float64(23.369999999999997), np.float64(20.630000000000003), np.float64(19.08), np.float64(17.68), np.float64(16.830000000000002), np.float64(16.21)], [np.float64(84.72), np.float64(62.22), np.float64(42.99), np.float64(30.98), np.float64(23.57), np.float64(19.259999999999998), np.float64(16.66), np.float64(14.729999999999999), np.float64(13.600000000000001), np.float64(12.64), np.float64(12.04)], [np.float64(85.